In [173]:
from selenium.webdriver import Chrome
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException, TimeoutException,NoSuchFrameException
import time
import pandas as pd

In [2]:
driver = Chrome()

In [3]:
url = 'https://search.naver.com/search.naver?where=post&sm=tab_jum&query=%ED%94%84%EB%A1%9C%EC%95%84%EC%9D%B4%ED%8C%94%EB%A0%88%ED%8A%B8&nso='
driver.get(url)

## 게시글 url

In [93]:
url_list = []
title_list = []
date_list = []
for i in range(1,991,10):
    url = 'https://search.naver.com/search.naver?date_from=20190213&date_option=8&date_to=20190801&dup_remove=1&nso=p%3Afrom20190213to20190801&post_blogurl=&post_blogurl_without=&query=%ED%81%B4%EB%A6%AC%EC%98%A4%20%ED%94%84%EB%A1%9C%EC%95%84%EC%9D%B4%ED%8C%94%EB%A0%88%ED%8A%B8&sm=tab_pge&srchby=all&st=sim&where=post&start='+str(i)
    driver.get(url) #페이지 접속
    
    urls = driver.find_elements_by_css_selector('a.url') #게시글 url    
    for url in urls:
        url_list.append(url.get_attribute('href'))
    
    titles = driver.find_elements_by_css_selector('a.sh_blog_title._sp_each_url._sp_each_title') #제목
    for title in titles:
        title_list.append(title.text)
    
    dates = driver.find_elements_by_css_selector('dd.txt_inline') #날짜
    for date in dates:
        date_list.append(date.text)

In [100]:
tot_list = []
tot_list.append([url_list, title_list, date_list])

In [106]:
proEye = pd.DataFrame({'url': url_list, '제목': title_list, '날짜': date_list})
proEye.head()

,url,제목,날짜
0,https://blog.naver.com/lqud?Redirect=Log&logNo...,클리오 프로 아이팔레트 03 코랄 토크 발색 및 후기,2019.04.26.
1,https://blog.naver.com/youridalbit?Redirect=Lo...,"클리오 프로 아이 팔레트 내돈내산 후기, 심플리핑크 아이메이...",2019.03.12.
2,https://blog.naver.com/ekejaqlsek?Redirect=Log...,"쿨톤 인생팔레트, 클리오 프로 아이팔레트 01 심플리핑크",6일 전
3,https://mythya0730.blog.me/221514170072,클리오 심플리핑크 프로 아이 팔레트 보이면 꼭 사세요! 품절 대란,2019.04.15.
4,https://blog.naver.com/hoyanim2?Redirect=Log&l...,클리오 프로 아이 팔레트 코랄토크 섀도우 색상 상세후기,2019.04.20.


In [112]:
proEye.to_excel('[클리오] 프로아이팔레트 블로그 후기.xlsx')

In [113]:
## 리스트 읽기
proEye = pd.read_excel('[클리오] 프로아이팔레트 블로그 후기.xlsx',  index_col=0)
proEye.head()

,url,제목,날짜
0,https://blog.naver.com/lqud?Redirect=Log&logNo...,클리오 프로 아이팔레트 03 코랄 토크 발색 및 후기,2019.04.26.
1,https://blog.naver.com/youridalbit?Redirect=Lo...,"클리오 프로 아이 팔레트 내돈내산 후기, 심플리핑크 아이메이...",2019.03.12.
2,https://blog.naver.com/ekejaqlsek?Redirect=Log...,"쿨톤 인생팔레트, 클리오 프로 아이팔레트 01 심플리핑크",6일 전
3,https://mythya0730.blog.me/221514170072,클리오 심플리핑크 프로 아이 팔레트 보이면 꼭 사세요! 품절 대란,2019.04.15.
4,https://blog.naver.com/hoyanim2?Redirect=Log&l...,클리오 프로 아이 팔레트 코랄토크 섀도우 색상 상세후기,2019.04.20.


## 게시글 긁기

In [185]:
contents_list = []
for i in range(486, len(proEye)):
    try:
        driver.get(proEye.iloc[i,0])
        time.sleep(3)

        ################# Case 1 ###################
        try:
            driver.switch_to.frame('mainFrame')
            time.sleep(1.5)
            nickname = driver.find_element_by_css_selector('#nickNameArea').text # 닉네임

            # 댓글이 있을 경우
            try: 
                replyCount = driver.find_element_by_css_selector('#printPost1 > tbody > tr > td.bcc > div.post-btn.post_btn2 > div.wrap_postcomment > div.area_comment.pcol2').text
                try: # 1.공감이 있을 경우
                    gongGamCount = driver.find_element_by_css_selector('em.u_cnt._count').text # 공감수
                    try: # 댓글이 있을 경우
                        driver.find_element_by_css_selector('span.btn_arr').click() # 댓글 더보기 클릭
                        time.sleep(2)
                        replys = driver.find_elements_by_css_selector('div.u_cbox_area') # 댓글
                        reply_text = ' / '.join([ reply.text for reply in replys ])
                        contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))
                    except: # 댓글이 없을 경우
                        reply_text = '댓글 없음'
                        contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))
                except: # 2.공감이 없을 경우
                    gongGamCount = '공감 없음'
                    try: # 댓글이 있을 경우
                        driver.find_element_by_css_selector('span.btn_arr').click() # 댓글 더보기 클릭
                        time.sleep(2)
                        replys = driver.find_elements_by_css_selector('div.u_cbox_area') # 댓글
                        reply_text = ' / '.join([ reply.text for reply in replys ])
                        contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))
                    except: # 댓글이 없을 경우
                        reply_text = '댓글 없음'
                        contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))

            # 댓글이 없을 경우
            except:
                replyCount = '댓글 없음'
                reply_text = '댓글 없음'
                try: # 1.공감이 있을 경우
                    gongGamCount = driver.find_element_by_css_selector('em.u_cnt._count').text # 공감수               
                    contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))           
                except: # 2.공감이 없을 경우
                    gongGamCount = '공감 없음'
                    contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))  

        ################# Case 2 ###################        
        except NoSuchFrameException:
            pass

        ################ Case 3 ####################
        except:
            driver.switch_to.frame('screenFrame')
            driver.switch_to.frame('mainFrame')
            time.sleep(1.5)        

            nickname = driver.find_element_by_css_selector('#nickNameArea').text # 닉네임

            # 댓글이 있을 경우
            try: 
                replyCount = driver.find_element_by_css_selector('#printPost1 > tbody > tr > td.bcc > div.post-btn.post_btn2 > div.wrap_postcomment > div.area_comment.pcol2').text
                try: # 1.공감이 있을 경우
                    gongGamCount = driver.find_element_by_css_selector('em.u_cnt._count').text # 공감수
                    try: # 댓글이 있을 경우
                        driver.find_element_by_css_selector('span.btn_arr').click() # 댓글 더보기 클릭
                        time.sleep(2)
                        replys = driver.find_elements_by_css_selector('div.u_cbox_area') # 댓글
                        reply_text = ' / '.join([ reply.text for reply in replys ])
                        contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))
                    except: # 댓글이 없을 경우
                        reply_text = '댓글 없음'
                        contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))
                except: # 2.공감이 없을 경우
                    gongGamCount = '공감 없음'
                    try: # 댓글이 있을 경우
                        driver.find_element_by_css_selector('span.btn_arr').click() # 댓글 더보기 클릭
                        time.sleep(2)
                        replys = driver.find_elements_by_css_selector('div.u_cbox_area') # 댓글
                        reply_text = ' / '.join([ reply.text for reply in replys ])
                        contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))
                    except: # 댓글이 없을 경우
                        reply_text = '댓글 없음'
                        contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))

            # 댓글이 없을 경우
            except:
                replyCount = '댓글 없음'
                reply_text = '댓글 없음'
                try: # 1.공감이 있을 경우
                    gongGamCount = driver.find_element_by_css_selector('em.u_cnt._count').text # 공감수               
                    contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))           
                except: # 2.공감이 없을 경우
                    gongGamCount = '공감 없음'
                    contents_list.append((proEye.iloc[i,0],nickname, gongGamCount, replyCount,reply_text))  
    except TimeoutException:
        pass

KeyboardInterrupt: 

In [186]:
len(contents_list)

328

In [187]:
contents_list[327]

('https://blog.naver.com/fastlane1301?Redirect=Log&logNo=221598830091',
 '츄츄트레인',
 '0',
 '댓글쓰기',
 '')

In [188]:
proEye[proEye['url']=='https://blog.naver.com/fastlane1301?Redirect=Log&logNo=221598830091']

,url,제목,날짜
886,https://blog.naver.com/fastlane1301?Redirect=L...,"[쿠팡] [뷰티] 클리오 프로 아이 팔레트, 2호 브라운슈, 1개",3일 전


In [189]:
con = pd.DataFrame(contents_list, columns = ['url', '닉네임', '공감수', '댓글수', '댓글'])
con.head()

,url,닉네임,공감수,댓글수,댓글
0,https://blog.naver.com/supersy_1?Redirect=Log&...,켈리엄,30,댓글 16,댓글이 삭제되었습니다. / 비밀 댓글입니다.\n2019.3.29. 12:49 / 비...
1,https://blog.naver.com/mane7256?Redirect=Log&l...,마내,28,댓글 2,동동이데디\n잘보고갑니다 ~^^ 즐거운하루되세요 ~^^\n2019.5.27. 04:...
2,https://blog.naver.com/sunchorong?Redirect=Log...,도토리,5,댓글쓰기,
3,https://blog.naver.com/urbane1027?Redirect=Log...,Imblackbaby,6,댓글 2,비밀 댓글입니다.\n2019.3.5. 00:15 / 비밀 댓글입니다.\n2019.3...
4,https://blog.naver.com/corudtmdfl1998?Redirect...,채롱,2,댓글쓰기,


In [190]:
con.to_excel('프로아이팔레트478_886.xlsx')